In [1]:

from langchain_chroma.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from openevals.llm import create_llm_as_judge
from openevals.prompts import RETRIEVAL_HELPFULNESS_PROMPT



In [2]:
# Initialize embeddings model (ensure the same model used during creation)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the existing Chroma vector store
persist_directory = "../db/swim_rules_semantic"
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)




## Helper Functions


In [ ]:

# Function to perform semantic search
def semantic_search(query, k=20):
    """
    Perform a semantic search on the Chroma vector store.

    Args:
        query (str): The query string to search for.
        k (int): The number of top results to retrieve.

    Returns:
        list: A list of documents matching the query.
    """
    results = vector_store.similarity_search_with_score(query, k=k,)
    chunks = [{"title": f"chunk_{chunk+1}", "content": result[0].page_content} for chunk, result in enumerate(results)]

    relevance = chunk_evaluator(inputs=query, outputs=chunks)
    print(f">>>Relevance: {relevance['key']} {relevance['score']},\n{relevance['comment']}")

    # print(f">>>Chunks:")
    # for chunk in chunks:
    #     print(f"Title: {chunk['title']}")
    #     print(f"Content: {chunk['content']}")
        # print("-" * 20)




chunk_evaluator = create_llm_as_judge(
    prompt=RETRIEVAL_HELPFULNESS_PROMPT,
    feedback_key="retrieval_helpfulness",
    model="openai:gpt-4o",
)




## Test chunk retrieval


In [4]:

# Example usage
queries = [
    "Prior to the start signal the swimmer is seen rocking back and forth.  Is this a disqualification?",
    "upon the command 'take your mark', the swimmer takes their starting position and just prior to the start signal, the swimmer is seen moving forward.",
    "What happens if a swimmer touches the wall with one hand in freestyle?",
    "what happens if a swimmer touches the wall with one hand in breaststroke?",
    "A swimmer is wearing a wetsuit in a swimming competition?",
    "During a freestyle event, a swimmer does a flip turn and touches the wall with only one foot.",
    "A breaststroke swimmer moves their hands in a sculling or flipper movement at the end of the first arm stroke, both after the start and after the turn Should they be disqualified?",
    "In a 9-10 100 yard breaststroke event, a swimmer completes 50 yards with a simulatenous two-hand touch and, thinking that the race is over, pushes back from the wall to read the scoreboard. At this point, realizing that the race is only halfway over, the swimmer returns to the wall, pushes off on the breast, and completes the required distance in good form. What call, if any, should be made?",
    "A swimmer swims the breaststroke in a way that their hands are completely underwater when they are pushed forward together from their breast.",
    "A swimmer swims the breaststroke in a way that their elbows are above the water line during the forward movement of the arms.",
    "A breaststroker’s head breaks the surface of the water during each cycle, however, the swimmer does not take a breath even if the head breaks the surface.",
    "During a freestyle event, a swimmer starts in the water such that the swimmwer is facing the starting end of the pool.",
    "During a freestyle event, a swimmer starts in the water such that the swimmer is facing away from the starting block.",
    "During a butterfly event, a swimmer starts in the water such that the swimmer is facing away from the starting block.",
    "In a 200m individual medley event, a swimmer swims the first 50m in butterfly, the second 50m in backstroke, the third 50m starts swimming freestyle but then switches to breaststroke, and the last 50m in freestyle.",
    "In a 100y freestyle event, the swimmer swims the first 50y in butterfly and the last 50y in backstroke.",
]

for query in queries[-4:]:
    print(f"\nQuery: {query}")
    print(semantic_search(query))





Query: During a freestyle event, a swimmer starts in the water such that the swimmer is facing away from the starting block.
>>>Relevance: retrieval_helpfulness True,
First, analyze the input: a freestyle swim event where the swimmer starts in the water facing away from the starting block. Key points include: the legality and specifications of an in-water start, and freestyle rules.

1. **Relevant Context**:
   - Freestyle swimming rules and requirements.
   - Rules regarding the swimmer's starting position, especially concerning 'in-water' start procedures.

2. **Analysis of the Retrieved Outputs**:
    
   - **Chunk 1**: Relevant as it specifies an in-water forward start is permitted for freestyle, noting the swimmer must have a hand in contact with the wall. This matches the input condition of starting in water.

   - **Chunk 2**: Irrelevant for this input as it primarily discusses general swimming strokes and race completion procedures not specifically aligned with in-water start 